##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Aprendizaje federado para generación de textos

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/federated/tutorials/federated_learning_for_text_generation"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/federated/tutorials/federated_learning_for_text_generation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/federated/tutorials/federated_learning_for_text_generation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/federated/tutorials/federated_learning_for_text_generation.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a>
</td>
</table>

**NOTA**: Esta colaboración ha sido verificada para trabajar con la [versión de lanzamiento más reciente](https://github.com/tensorflow/federated#compatibility) del paquete pip `tensorflow_federated`, pero el proyecto federado de TensorFlow aún se encuentra en una etapa de desarrollo previa al lanzamiento. Por lo tanto, es probable que no funcione en `main`.

Este tutorial se desarrolla en base a los conceptos del tutorial sobre [aprendizaje federado para clasificación de imágenes](federated_learning_for_image_classification.ipynb), y aquí se demuestran muchos otros métodos útiles  de aplicación del aprendizaje federado.

En particular, cargamos un modelo keras previamente entrenado y lo refinamos con el aprendizaje federado de un conjunto de datos (simulado) descentralizado. En la práctica resulta importante por varios motivos. La posibilidad de usar modelos serializados facilita la posibilidad de mezclar aprendizaje federado con otros métodos de aprendizaje automático. Además, permite usar un rango más amplio de modelos previamente entrenados. Por ejemplo, los modelos de lenguaje de entrenamiento desde cero rara vez son necesarios, ya que hoy en día hay muchos modelos previamente entrenados disponibles para usar (consulte, p. ej., [TF Hub](https://www.tensorflow.org/hub)).<br>Tiene más sentido empezar a partir de un modelo previamente entrenado y refinarlo con aprendizaje federado, para adaptarlo a las características particulares de los datos descentralizados para una aplicación particular.

Para este tutorial, empezamos con una RNN que genera caracteres ASCII y lo refinamos mediante aprendizaje federado. También mostramos de qué manera los pesos finales se pueden volver a incorporar al modelo Keras original, facilitando la evaluación y la generación de textos con herramientas estándares.

In [ ]:
#@test {"skip": true}
!pip install --quiet --upgrade tensorflow-federated

In [ ]:
import collections
import functools
import os
import time

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

np.random.seed(0)

# Test that TFF is working:
tff.federated_computation(lambda: 'Hello, World!')()

b'Hello, World!'

## Carga de un modelo previamente entrenado

Cargamos un modelo que fue previamente entrenado siguiendo el tutorial sobre [Generación de textos con una RNN mediante ejecución <em>eager</em>](https://www.tensorflow.org/tutorials/sequences/text_generation) de TensorFlow. Sin embargo, más que entrenar [La obra completa de Shakespeare](http://www.gutenberg.org/files/100/100-0.txt), preentrenamos el modelo con los textos de [Historia de dos ciudades](http://www.ibiblio.org/pub/docs/books/gutenberg/9/98/98.txt) y [Canción de Navidad](http://www.ibiblio.org/pub/docs/books/gutenberg/4/46/46.txt) de Charles Dickens.

Si bien expandimos el vocabulario, no modificamos el tutorial original; por lo tanto, este modelo inicial no es de lo más avanzado, pero produce predicciones razonables y es adecuado para cumplir con el objetivo del tutorial. El modelo final fue guardado en `tf.keras.models.save_model(include_optimizer=False)`.

Utilizaremos aprendizaje federado para refinar el modelo para trabajar con Shakespeare en este tutorial. Usaremos una versión federada de los datos provistos por TFF.


### Generación de las tablas para búsqueda de vocabulario

In [ ]:
# A fixed vocabularly of ASCII chars that occur in the works of Shakespeare and Dickens:
vocab = list('dhlptx@DHLPTX $(,048cgkoswCGKOSW[_#\'/37;?bfjnrvzBFJNRVZ"&*.26:\naeimquyAEIMQUY]!%)-159\r')

# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

### Carga del modelo previamente entrenado y generación de algo de texto

In [ ]:
def load_model(batch_size):
  urls = {
      1: 'https://storage.googleapis.com/tff-models-public/dickens_rnn.batch1.kerasmodel',
      8: 'https://storage.googleapis.com/tff-models-public/dickens_rnn.batch8.kerasmodel'}
  assert batch_size in urls, 'batch_size must be in ' + str(urls.keys())
  url = urls[batch_size]
  local_file = tf.keras.utils.get_file(os.path.basename(url), origin=url)  
  return tf.keras.models.load_model(local_file, compile=False)

In [ ]:
def generate_text(model, start_string):
  # From https://www.tensorflow.org/tutorials/sequences/text_generation
  num_generate = 200
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  temperature = 1.0

  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(
        predictions, num_samples=1)[-1, 0].numpy()
    input_eval = tf.expand_dims([predicted_id], 0)
    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
# Text generation requires a batch_size=1 model.
keras_model_batch1 = load_model(batch_size=1)
print(generate_text(keras_model_batch1, 'What of TensorFlow Federated, you ask? '))

16193984/16193984 [==============================] - 0s 0us/step
What of TensorFlow Federated, you ask? Same yee you? Have I so,
often games in a man who rode one knee over his friend, with the
stone faces of the dread prisoners, dud a tender mastery. They
are not alive is infirmed us--to ever resume


## Carga y procesamiento previo de los datos federados de Shakespeare

El paquete `tff.simulation.datasets` ofrece una variedad de conjuntos de datos separados en los "clientes", donde a cada cliente le corresponde un conjunto de datos en un dispositivo en particular que podría participar en el aprendizaje federado.

Estos conjuntos de datos muestran distribuciones realistas de datos no independientes (<em>non-IID</em>), que replican en simulación las dificultades del entrenamiento en datos reales descentralizados. Parte de este procesamiento previo de los datos se realizó con herramientas del [proyecto Leaf](https://arxiv.org/abs/1812.01097) ([github](https://github.com/TalwalkarLab/leaf)).

In [ ]:
train_data, test_data = tff.simulation.datasets.shakespeare.load_data()

Los conjuntos de datos provistos por `shakespeare.load_data()` están compuestos por una secuencia de cadena de `Tensors`, una para cada línea hablada por uno de los personajes de una obra de Shakespeare. Las claves de cliente están formadas por el nombre de la obra y el nombre del personaje; por ejemplo `MUCH_ADO_ABOUT_NOTHING_OTHELLO` corresponde a las líneas del personaje Otelo en la obra *Mucho ruido y pocas nueces*. Tenga en cuenta que en un escenario de aprendizaje federado a los clientes nunca se los identifica ni se los rastrea por ID, pero para la simulación resulta útil trabajar con conjuntos de datos codificados (con clave).

Aquí, por ejemplo, podemos observar algunos datos de El rey Lear:

In [ ]:
# Here the play is "The Tragedy of King Lear" and the character is "King".
raw_example_dataset = train_data.create_tf_dataset_for_client(
    'THE_TRAGEDY_OF_KING_LEAR_KING')
# To allow for future extensions, each entry x
# is an OrderedDict with a single key 'snippets' which contains the text.
for x in raw_example_dataset.take(2):
  print(x['snippets'])

tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'What?', shape=(), dtype=string)


Ahora usamos transformaciones `tf.data.Dataset` para preparar estos datos para el entrenamiento en la RNN cargada anteriormente.


In [ ]:
# Input pre-processing parameters
SEQ_LENGTH = 100
BATCH_SIZE = 8
BUFFER_SIZE = 100  # For dataset shuffling

In [ ]:
# Construct a lookup table to map string chars to indexes,
# using the vocab loaded above:
table = tf.lookup.StaticHashTable(
    tf.lookup.KeyValueTensorInitializer(
        keys=vocab, values=tf.constant(list(range(len(vocab))),
                                       dtype=tf.int64)),
    default_value=0)


def to_ids(x):
  s = tf.reshape(x['snippets'], shape=[1])
  chars = tf.strings.bytes_split(s).values
  ids = table.lookup(chars)
  return ids


def split_input_target(chunk):
  input_text = tf.map_fn(lambda x: x[:-1], chunk)
  target_text = tf.map_fn(lambda x: x[1:], chunk)
  return (input_text, target_text)


def preprocess(dataset):
  return (
      # Map ASCII chars to int64 indexes using the vocab
      dataset.map(to_ids)
      # Split into individual chars
      .unbatch()
      # Form example sequences of SEQ_LENGTH +1
      .batch(SEQ_LENGTH + 1, drop_remainder=True)
      # Shuffle and form minibatches
      .shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
      # And finally split into (input, target) tuples,
      # each of length SEQ_LENGTH.
      .map(split_input_target))

Considere que para facilitar la formación de las secuencias originales y la de los lotes que figuran aquí arriba usamos `drop_remainder=True`. Significa que cualquiera de los personajes (clientes) que no tienen al menos `(SEQ_LENGTH + 1) * BATCH_SIZE` caracteres de texto tendrán bases de datos vacías. Una forma muy común de abordarlo sería agrupando los lotes con un token especial y enmascarando después la pérdida para que no se tengan en cuenta los tokens de agrupamiento (<em>padding</em>).

En cierto modo, esto haría que el ejemplo fuera más complicado. Entonces, para este tutorial solamente usamos lotes completos, como en el [tutorial estándar](https://www.tensorflow.org/tutorials/sequences/text_generation). Sin embargo, en el ambiente federado este problema se torna más significativo, porque puede haber muchos usuarios con pequeños conjuntos de datos.

Ahora podemos procesar nuestro `raw_example_dataset` y verificar los tipos:

In [ ]:
example_dataset = preprocess(raw_example_dataset)
print(example_dataset.element_spec)

(TensorSpec(shape=(8, 100), dtype=tf.int64, name=None), TensorSpec(shape=(8, 100), dtype=tf.int64, name=None))


## Compilación del modelo y comprobación de los datos preprocesados

Cargamos un modelo keras sin compilar, pero para ejecutar `keras_model.evaluate` necesitamos compilarlo con una pérdida y métricas. También compilamos un optimizador, que se usará como optimizador en dispositivos, como parte del aprendizaje federado.

El tutorial original no tiene una precisión a nivel de caracteres (la fracción de predicciones en que la probabilidad más alta se asignó al siguiente carácter correcto). Es una métrica útil, así que la agregamos. Sin embargo, debemos definir una clase de métrica nueva porque nuestras predicciones tienen un nivel 3 (un vector de funciones <em>logit</em> para cada predicción `BATCH_SIZE * SEQ_LENGTH`) y `SparseCategoricalAccuracy` espera solamente predicciones de nivel 2.

In [ ]:
class FlattenedCategoricalAccuracy(tf.keras.metrics.SparseCategoricalAccuracy):

  def __init__(self, name='accuracy', dtype=tf.float32):
    super().__init__(name, dtype=dtype)

  def update_state(self, y_true, y_pred, sample_weight=None):
    y_true = tf.reshape(y_true, [-1, 1])
    y_pred = tf.reshape(y_pred, [-1, len(vocab), 1])
    return super().update_state(y_true, y_pred, sample_weight)

Ahora podemos compilar un modelo y evaluarlo en nuestro `example_dataset`.

In [ ]:
BATCH_SIZE = 8  # The training and eval batch size for the rest of this tutorial.
keras_model = load_model(batch_size=BATCH_SIZE)
keras_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[FlattenedCategoricalAccuracy()])

# Confirm that loss is much lower on Shakespeare than on random data
loss, accuracy = keras_model.evaluate(example_dataset.take(5), verbose=0)
print(
    'Evaluating on an example Shakespeare character: {a:3f}'.format(a=accuracy))

# As a sanity check, we can construct some completely random data, where we expect
# the accuracy to be essentially random:
random_guessed_accuracy = 1.0 / len(vocab)
print('Expected accuracy for random guessing: {a:.3f}'.format(
    a=random_guessed_accuracy))
random_indexes = np.random.randint(
    low=0, high=len(vocab), size=1 * BATCH_SIZE * (SEQ_LENGTH + 1))
data = collections.OrderedDict(
    snippets=tf.constant(
        ''.join(np.array(vocab)[random_indexes]), shape=[1, 1]))
random_dataset = preprocess(tf.data.Dataset.from_tensor_slices(data))
loss, accuracy = keras_model.evaluate(random_dataset, steps=10, verbose=0)
print('Evaluating on completely random data: {a:.3f}'.format(a=accuracy))

16193984/16193984 [==============================] - 0s 0us/step
Evaluating on an example Shakespeare character: 0.45.000
Expected accuracy for random guessing: 0.012
Evaluating on completely random data: 0.011


## Ajuste fino del modelo con aprendizaje federado

TFF serializa todos los cálculos de TensorFlow para que se puedan ejecutar potencialmente en un entorno que no sea de Python (a pesar de que en este momento, solamente hay un tiempo de ejecución de simulación implementado en Python). Si bien la ejecución se hace en modo <em>eager</em>, (TF 2.0), actualmente TFF serializa los cálculos de TensorFlow construyendo las operaciones necesarias dentro del contexto de una afirmación "`with tf.Graph.as_default()`". Por lo tanto, debemos aportar una función que TFF pueda usar para introducir nuestro modelo en un grafo que controle. Lo hacemos de la siguiente manera:

In [ ]:
# Clone the keras_model inside `create_tff_model()`, which TFF will
# call to produce a new copy of the model inside the graph that it will 
# serialize. Note: we want to construct all the necessary objects we'll need 
# _inside_ this method.
def create_tff_model():
  # TFF uses an `input_spec` so it knows the types and shapes
  # that your model expects.
  input_spec = example_dataset.element_spec
  keras_model_clone = tf.keras.models.clone_model(keras_model)
  return tff.learning.models.from_keras_model(
      keras_model_clone,
      input_spec=input_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[FlattenedCategoricalAccuracy()])

Ahora estamos listos para construir un proceso iterativo de cálculo de promedios federados que luego usaremos para mejorar el modelo (para más detalles sobre el algoritmo de cálculo de promedio federado, consulte la publicación [Communication-Efficient Learning of Deep Networks from Decentralized Data](https://arxiv.org/abs/1602.05629) (Aprendizaje con comunicaciones eficientes de redes profundas a partir de datos descentralizados).

Usamos un modelo Keras para realizar una evaluación estándar (no federada) después de cada ronda de entrenamiento federado. A los fines investigativos, resulta útil cuando hacemos aprendizaje federado simulado con un conjunto de datos de prueba estándar.

En un entorno de producción realista esta misma técnica se podría utilizar para tomar modelos entrenados con aprendizaje federado y evaluarlos en un conjunto de datos centralizado de referencia para realizar las pruebas o para aseguramiento de la calidad.

In [ ]:
# This command builds all the TensorFlow graphs and serializes them: 
fed_avg = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn=create_tff_model,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.5))

El siguiente es el ciclo más simple posible, en él ejecutamos el cálculo de promedio federado para una ronda en un solo cliente de un solo lote:

In [ ]:
state = fed_avg.initialize()
result = fed_avg.next(state, [example_dataset.take(5)])
state = result.state
train_metrics = result.metrics['client_work']['train']
print('loss={l:.3f}, accuracy={a:.3f}'.format(
    l=train_metrics['loss'], a=train_metrics['accuracy']))

loss=4.399, accuracy=0.139


Ahora escribamos un ciclo de entrenamiento y evaluación un poco más interesante.

Para que esta simulación todavía se ejecute con rapidez, llevaremos a cabo el entrenamiento en los mismos tres clientes cada vez, considerando solamente dos minilotes para cada una.


In [ ]:
def data(client, source=train_data):
  return preprocess(source.create_tf_dataset_for_client(client)).take(5)


clients = [
    'ALL_S_WELL_THAT_ENDS_WELL_CELIA', 'MUCH_ADO_ABOUT_NOTHING_OTHELLO',
]

train_datasets = [data(client) for client in clients]

# We concatenate the test datasets for evaluation with Keras by creating a 
# Dataset of Datasets, and then identity flat mapping across all the examples.
test_dataset = tf.data.Dataset.from_tensor_slices(
    [data(client, test_data) for client in clients]).flat_map(lambda x: x)

El estado inicial del modelo producido por `fed_avg.initialize()` se basa en los inicializadores aleatorios para el modelo Keras, no en los pesos que se cargaron, ya que `clone_model()` no clona los pesos. Para empezar el entrenamiento a partir de un modelo previamente entrenado, configuramos los pesos del modelo en el estado del servidor directamente a partir del modelo cargado.

In [ ]:
NUM_ROUNDS = 5

# The state of the FL server, containing the model and optimization state.
state = fed_avg.initialize()

# Load our pre-trained Keras model weights into the global model state.
pre_trained_weights = tff.learning.models.ModelWeights(
    trainable=[v.numpy() for v in keras_model.trainable_weights],
    non_trainable=[v.numpy() for v in keras_model.non_trainable_weights]
)
state = fed_avg.set_model_weights(state, pre_trained_weights)


def keras_evaluate(state, round_num):
  # Take our global model weights and push them back into a Keras model to
  # use its standard `.evaluate()` method.
  keras_model = load_model(batch_size=BATCH_SIZE)
  keras_model.compile(
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[FlattenedCategoricalAccuracy()])
  model_weights = fed_avg.get_model_weights(state)
  model_weights.assign_weights_to(keras_model)
  loss, accuracy = keras_model.evaluate(example_dataset, steps=2, verbose=0)
  print('\tEval: loss={l:.3f}, accuracy={a:.3f}'.format(l=loss, a=accuracy))


for round_num in range(NUM_ROUNDS):
  print('Round {r}'.format(r=round_num))
  keras_evaluate(state, round_num)
  result = fed_avg.next(state, train_datasets)
  state = result.state
  train_metrics = result.metrics['client_work']['train']
  print('\tTrain: loss={l:.3f}, accuracy={a:.3f}'.format(
      l=train_metrics['loss'], a=train_metrics['accuracy']))

print('Final evaluation')
keras_evaluate(state, NUM_ROUNDS + 1)

Round 0
	Eval: loss=3.171, accuracy=0.428
	Train: loss=4.309, accuracy=0.098
Round 1
	Eval: loss=4.188, accuracy=0.185
	Train: loss=4.037, accuracy=0.223
Round 2
	Eval: loss=3.948, accuracy=0.200
	Train: loss=3.797, accuracy=0.228
Round 3
	Eval: loss=3.826, accuracy=0.179
	Train: loss=3.662, accuracy=0.219
Round 4
	Eval: loss=3.723, accuracy=0.171
	Train: loss=3.440, accuracy=0.245
Final evaluation
	Eval: loss=3.599, accuracy=0.181


Con los cambios predeterminados, el entrenamiento no ha sido suficiente como para lograr una gran diferencia. Pero si lo entrenamos por más tiempo y con más datos de Shakespeare deberíamos notar una diferencia en el estilo del texto generado con el modelo actualizado:

In [ ]:
# Set our newly trained weights back in the originally created model.
keras_model_batch1.set_weights([v.numpy() for v in keras_model.weights])
# Text generation requires batch_size=1
print(generate_text(keras_model_batch1, 'What of TensorFlow Federated, you ask? '))

What of TensorFlow Federated, you ask? She will be
heard of; or whether they recovered her faltering place, that a great mark of
being so final dark and distrustner the dearer to the chin, all
staftly towards him, or trot's in foot thro


## Extensiones sugeridas

En este tutorial se muestra solamente el primer paso. A continuación, compartimos algunas ideas sobre cómo ampliar lo compartido en estas notas:

- Escriba un circuito de entrenamiento más realista en el que se haga entrenamiento aleatorio con clientes de muestra.
- Use "`.repeat(NUM_EPOCHS)`" de los conjuntos de datos del cliente para intentar multiplicar las épocas de entrenamiento focal (p. ej., como en [McMahan et. al.](https://arxiv.org/abs/1602.05629)). Consulte también [Aprendizaje federado para clasificación de imágenes](federated_learning_for_image_classification.ipynb), que lo hace.
- Cambie el comando `compile()` para experimentar con diferentes algoritmos de optimización en el cliente.
- Pruebe el argumento `server_optimizer` para `build_weighted_fed_avg` con el objetivo de probar algoritmos diferentes para aplicar las actualizaciones del modelo en el servidor.